In [80]:
import pandas as pd
import numpy as np
from pathlib import Path

In [81]:
file_names = [
    "listings_202309.csv",
    "listings_202312.csv",
    "listings_202403.csv",
    "listings_202406.csv",
]

In [82]:
all_listings = pd.concat(
    (
        pd.read_csv(f)
        for f in [Path(f"../local_data/{file_name}") for file_name in file_names]
    ),
    ignore_index=True,
).sort_values("last_scraped")

In [69]:
dates = all_listings["last_scraped"].unique()


for date in dates:
    temp_df = all_listings[all_listings["last_scraped"] == date].dropna(
        axis=1, how="all"
    )
    print(
        f"Date: {date}: cols missing: {[ele for ele in all_listings.columns.tolist() if ele not in temp_df.columns.tolist()]}"
    )

all_listings["last_scraped"] = all_listings["last_scraped"].replace(
    "2023-12-27", "2023-12-28"
)
all_listings = all_listings.dropna(subset=["price"], axis=0, how="all")
all_listings["bathrooms"] = np.where(
    all_listings["bathrooms_text"].isna(),
    all_listings["bathrooms"],
    all_listings["bathrooms_text"],
)

Date: 2023-09-24: cols missing: ['bathrooms', 'calendar_updated', 'license']
Date: 2023-12-27: cols missing: ['description', 'bathrooms', 'bedrooms', 'calendar_updated', 'license']
Date: 2023-12-28: cols missing: ['description', 'bathrooms', 'bedrooms', 'calendar_updated', 'license']
Date: 2024-03-31: cols missing: ['calendar_updated', 'license']
Date: 2024-06-29: cols missing: ['calendar_updated', 'license']


In [70]:
all_listings.groupby("last_scraped").size()

last_scraped
2023-09-24    2534
2023-12-28    2372
2024-03-31    2140
2024-06-29    2323
dtype: int64

In [71]:
pd.set_option("display.max_columns", None)

In [72]:
a = all_listings.groupby("last_scraped").count().sort_index()
a.loc[:, ((a == 0).sum() > 1)]

,calendar_updated,license
last_scraped,,
2023-09-24,0,0
2023-12-28,0,0
2024-03-31,0,0
2024-06-29,0,0


In [73]:
all_listings.filter(["bedrooms", "beds"]).describe()

,bedrooms,beds
count,6192.000000,9257.000000
mean,1.257752,1.553959
std,0.898196,1.151982
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,2.000000
max,32.000000,32.000000


In [74]:
all_listings["bathrooms_text"]

0               1 bath
1685     1 shared bath
1686            1 bath
1687     1 shared bath
1688            1 bath
             ...      
9176            1 bath
9177            1 bath
9178            1 bath
9179     1 shared bath
11182           1 bath
Name: bathrooms_text, Length: 9369, dtype: object

In [75]:
all_listings["beds"].fillna(np.nan).sub(all_listings["bedrooms"]).value_counts()

 0.0     3866
 1.0     1495
-1.0      342
 2.0      291
 3.0       56
-2.0       50
 4.0       29
 5.0       11
-3.0        8
 6.0        4
-4.0        3
 16.0       1
-19.0       1
-6.0        1
-5.0        1
Name: count, dtype: int64

In [76]:
min_entries = 2

keep_ids = (
    all_listings.value_counts("id").loc[lambda x: x >= min_entries].index.tolist()
)
relevant_listings = (
    all_listings[all_listings["id"].isin(keep_ids)]
    .drop_duplicates("id", keep="last")
    .dropna(how="all", axis=1)
)

In [77]:
relevant_listings.to_csv("../local_data/relevant_listings.csv", index=False)

In [78]:
relevant_listings.sort_values("price", ascending=False)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
6847,668839556377128266,https://www.airbnb.com/rooms/668839556377128266,20240330234249,2024-03-31,city scrape,Luxury Penthouse near the Dolder Hotel,Ever dreamed of living in a luxury penthouse? ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,40704806,https://www.airbnb.com/users/show/40704806,Florence,2015-08-06,"Zürich, Switzerland","Hello :) I am Florence, living in Zurich. I tr...",NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/fb493...,https://a0.muscache.com/im/pictures/user/fb493...,NaN,1,1,['phone'],t,f,NaN,Hirslanden,Kreis 7,47.364930,8.569090,Entire condo,Entire home/apt,4,2 baths,2 baths,2.0,2.0,"[""Hair dryer"", ""Free washer \u2013 In building...",$999.00,10,1125,10,10,1125,1125,10.0,1125.0,t,30,60,90,365,2024-03-31,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,1,1,0,0,NaN
7508,955784712671658608,https://www.airbnb.com/rooms/955784712671658608,20240330234249,2024-03-31,city scrape,Zurich City Apartment - Wollishofen,Wellcome in Zurich,NaN,https://a0.muscache.com/pictures/72f799fd-f7d1...,177523078,https://www.airbnb.com/users/show/177523078,Markus,2018-03-09,"Zürich, Switzerland",NaN,within a day,100%,79%,t,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,NaN,1,2,"['email', 'phone']",t,t,NaN,Wollishofen,Kreis 2,47.336297,8.531129,Entire rental unit,Entire home/apt,2,1 bath,1 bath,1.0,1.0,"[""Clothing storage: dresser"", ""Free washer \u2...",$99.00,3,15,1,3,15,15,3.0,15.0,t,20,40,46,46,2024-03-31,17,17,1,2023-09-28,2024-03-25,5.00,5.00,5.00,5.00,5.00,4.94,4.94,f,1,1,0,0,2.74
4796,937575075386386458,https://www.airbnb.com/rooms/937575075386386458,20231227173841,2023-12-28,city scrape,Rental unit in Zürich · ★4.55 · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,83233956,https://www.airbnb.com/users/show/83233956,Ariana,2016-07-12,"Zürich, Switzerland","Wer zusammen lacht, versteht sich. Wir erkenne...",within an hour,100%,97%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,NaN,1,1,"['email', 'phone']",t,t,NaN,Unterstrass,Kreis 6,47.400853,8.532059,Entire rental unit,Entire home/apt,2,1 bath,1 bath,NaN,1.0,[],$99.00,2,30,1,2,1125,1125,2.0,1125.0,t,22,28,38,38,2023-12-27,22,22,2,2023-08-04,2023-12-16,4.55,4.73,4.77,4.82,4.73,4.68,4.45,t,1,1,0,0,4.52
8483,15653745,https://www.airbnb.com/rooms/15653745,20240629155539,2024-06-29,city scrape,Nice 2.5 room Raspberry I Apartment,"The ""Raspberry I"" Business Apartment is the pe...",The modern business apartment is located close...,https://a0.muscache.com/pictures/d3861949-8acc...,12886487,https://www.airbnb.com/users/show/12886487,Stephanie,2014-03-07